In [ ]:
import pandas as pd
from collections import defaultdict
from gensim import corpora, models
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import ExtraTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.svm import LinearSVC, SVC
import numpy as np
from gensim.models import Phrases

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
from scripts.processing import *
from scripts.mk_categories_word2vec_addmaincat import *

RSEED = 42

In [ ]:
# import CSV
df = pd.read_csv('../data/yelp_dataset/review_1819.csv')
# maincat = select_dataset_by_cat()

In [ ]:
# df_maincat = df.reset_index().merge(maincat.reset_index(), on='business_id', how='left') # add suffixes…
# maincat_restr = ["beauty & spas", "fitness & instruction"]
# df_maincat = df_maincat.query('maincat == @maincat_restr')
# restricting the dataset to a subset of maincats does not seem to change much…

In [ ]:
# df_maincat.groupby('maincat').count().review_id

In [ ]:
# df_maincat = language_processing(df_maincat, verbose=True) 
df = language_processing(df, verbose=True) # why such results with some maincats?

In [ ]:
# https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html#sphx-glr-auto-examples-core-run-topics-and-transformations-py
# https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
documents = []
# for document in df_maincat.iloc[0:50000].text:
# for document in df_maincat.iloc[0:100000].text:
for document in df.iloc[0:100000].text:
# for document in df.iloc[0:1000000].text:
    documents.append(document)

stoplist = stopwords.words('english')

# TODO use different tokenizer and lemmatizer? (also elsewhere in this notebook)
# texts = [ [lemmatizer.lemmatize(token) for token in [ token for token in word_tokenize(document.translate(str.maketrans('', '', string.punctuation)).lower()) ] if token not in stoplist] for document in documents ]
texts = [ [ token for token in word_tokenize(document.lower()) if token not in stoplist] for document in documents ] # instead of stoplist use only no_above/below as below??

In [ ]:
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html
# Remove numbers, but not words that contain numbers.
texts = [[token for token in doc if not token.isnumeric()] for doc in texts]

# Remove words that are only one character.
texts = [[token for token in doc if len(token) > 1] for doc in texts]

In [ ]:
lemmatizer = WordNetLemmatizer()
texts = [ [lemmatizer.lemmatize(token) for token in doc] for doc in texts ]

In [ ]:
# (slightly adapted) from https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html
# Add bigrams and trigrams to docs (only ones that appear 20[now 10] times or more).
bigram = Phrases(texts, min_count=10) # TODO check if something can be improved (not really improving anything like this…)
for idx in range(len(texts)):
    for token in bigram[texts[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            texts[idx].append(token)

In [ ]:
# freq = defaultdict(int)
# for text in texts:
#     for token in text:
#         freq[token] += 1
# 
# texts = [
#     [token for token in text if freq[token] > 1]
#     for text in texts
# ]

In [ ]:
dictionary = corpora.Dictionary(texts)

# https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html
# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
# tfidf = models.TfidfModel(corpus)

In [ ]:
# corpus_tfidf = tfidf[corpus]

## LDA

In [ ]:
NUM_TOPICS = 40

In [ ]:
# lda_model = models.LdaMulticore(corpus_tfidf, id2word=dictionary, num_topics=NUM_TOPICS)
# lda_model = models.LdaMulticore(corpus_tfidf, id2word=dictionary, num_topics=NUM_TOPICS, passes=20, iterations=400, eval_every=1)
# lda_model = models.LdaMulticore(corpus_tfidf, id2word=dictionary, num_topics=NUM_TOPICS, passes=10, iterations=200, eval_every=1)
# lda_model = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=NUM_TOPICS, alpha='auto', eta='auto', passes=20, iterations=400) # cf. also https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html
# corpus_lda = lda_model[corpus]

lda_model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=NUM_TOPICS, passes=10, iterations=200, eval_every=1)

In [ ]:
# lda_model.print_topics()

In [ ]:
# df_topics = df.iloc[1100000:1200000].copy() # copy necessary?
df_topics = df.iloc[200000:300000].copy() # copy necessary?
# df_topics = df.iloc[1000000:].copy() # copy necessary?
# df_topics = df_maincat.iloc[200000:300000].copy() # copy necessary?
# df_topics = df_maincat.iloc[50000:100000].copy() # copy necessary?
for i in range(NUM_TOPICS):
    df_topics[f"topic_{i}"] = 0
    # df_topics[f"topic_{i}"] = 0.0

In [ ]:
for idx, row in df_topics.iterrows():
    unseen_doc_text = row.text
    unseen_doc_text_tok = [lemmatizer.lemmatize(token) for token in [ token for token in word_tokenize(unseen_doc_text.translate(str.maketrans('', '', string.punctuation)).lower()) ] if token not in stoplist]
    # unseen_doc_text_tok = [token for token in unseen_doc_text.translate(str.maketrans('', '', string.punctuation)).lower().split() if token not in stoplist]
    unseen_doc = dictionary.doc2bow(unseen_doc_text_tok)
    vector = lda_model[unseen_doc]
    # print(vector)
    for topic in vector:
        df_topics.loc[idx, f"topic_{topic[0]}"] = 1 if topic[1] > 0 else 0 # different threshold?
        # df_topics.loc[idx, f"topic_{topic[0]}"] = topic[1]

In [ ]:
# split data into feature and target 
# maincat_dummies = pd.get_dummies(df_topics.maincat, drop_first=True)
# X = pd.concat([maincat_dummies, df_topics.filter(like='topic')], axis=1)
# worse with main cat dummies…

X = df_topics.filter(like='topic')

In [ ]:
y = df_topics['useful'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
# split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RSEED, stratify=y)

In [ ]:
def mcc(cm):
    tn, fp = cm[0]
    fn, tp = cm[1]
    return (tp*tn-fp*fn) / ((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))**0.5

In [ ]:
# initialize the Classifier
logreg = LogisticRegression(max_iter=10000)

# fit the model
logreg.fit(X_train, y_train)    

# make predictions
y_pred = logreg.predict(X_test)

# test the model
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
print(mcc(confusion_matrix(y_test, y_pred)))

# show the classification report
print(classification_report(y_test, y_pred))

In [ ]:
coef = logreg.coef_[0]

for item in np.argsort(coef)[-1:-6:-1]:
    print(coef[item], lda_model.print_topic(item))
print("="*40)
for item in np.argsort(coef)[:5]:
    print(coef[item], lda_model.print_topic(item))

In [ ]:
param_logreg = {'penalty':('l1','l2', 'elasticnet'),#, None),
                'C': [9, 9.5, 10, 10.5, 11],
                'solver': ['liblinear', 'lbfgs', 'sag'],#, 'newton-cg', 'saga']
               }

# mcc_scorer = make_scorer(matthews_corrcoef)
grid_logreg = GridSearchCV(LogisticRegression(), param_grid=param_logreg, cv=5, scoring='precision',#mcc_scorer, 
                           verbose=5, n_jobs=-1)

# fit the model
grid_logreg.fit(X_train, y_train)    

# Show best parameters
print('Best score:\n{:.2f}'.format(grid_logreg.best_score_))
print("Best parameters:\n{}".format(grid_logreg.best_params_))

In [ ]:
y_pred = grid_logreg.best_estimator_.predict(X_test)

# test the model
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
print(mcc(confusion_matrix(y_test, y_pred)))
 
# show the classification report
print(classification_report(y_test, y_pred))

In [ ]:
# initialize the Classifier
LSVC = LinearSVC()

# fit the model
LSVC.fit(X_train, y_train)

# make predictions
y_pred = LSVC.predict(X_test)

# test the model
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
print(mcc(confusion_matrix(y_test, y_pred)))

# show the classification report
print(classification_report(y_test, y_pred))

In [ ]:
# LSVC.coef_

In [ ]:
aram_linsvc = {'penalty':('l1','l2'),
                'loss': ('hinge', 'squared_hinge'),
                'C': [0.05, 0.06, 0.065],
                'class_weight': ('balanced', None),
                'max_iter': [10000]
               }

# mcc_scorer = make_scorer(matthews_corrcoef)
grid_linsvc = GridSearchCV(LinearSVC(), param_grid=param_linsvc, cv=5, scoring='precision',#'recall',#mcc_scorer, 
                           verbose=5, n_jobs=-1)

# fit the model
grid_linsvc.fit(X_train, y_train)    

# Show best parameters
print('Best score:\n{:.2f}'.format(grid_linsvc.best_score_))
print("Best parameters:\n{}".format(grid_linsvc.best_params_))


In [ ]:
y_pred = grid_linsvc.best_estimator_.predict(X_test)

# test the model
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
print(mcc(confusion_matrix(y_test, y_pred)))
 
# show the classification report
print(classification_report(y_test, y_pred))

In [ ]:
# initialize the Classifier
xtree = ExtraTreeClassifier()

# fit the model
xtree.fit(X_train, y_train)    

# make predictions
y_pred = xtree.predict(X_test)

# test the model
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
print(mcc(confusion_matrix(y_test, y_pred)))

# show the classification report
print(classification_report(y_test, y_pred))

## Trying the separate script

In [ ]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
 
from scripts.topic_feature import *
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
 
import seaborn as sns
 
RSEED = 42

In [ ]:
NUM_TOPICS = 40
df = pd.read_csv('../data/yelp_dataset/review_1819.csv')
lda_model, dictionary = train_LDA_model(df, end=100000, num_topics=NUM_TOPICS)


In [ ]:
X = add_topics(df, lda_model, dictionary, begin=200000, end=300000, num_topics=NUM_TOPICS)

In [ ]:
df_merge = df.merge(X, on='review_id', how='left').fillna(0)
df_merge

In [ ]:
df_merge.iloc[199990:200010]

In [ ]:
df.iloc[200000:300000].index

In [ ]:
y = df.iloc[200000:300000]['useful'].apply(lambda x: 1 if x > 0 else 0)

# split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RSEED, stratify=y)

# def mcc(cm):
#     tn, fp = cm[0]
#     fn, tp = cm[1]
#     return (tp*tn-fp*fn) / ((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))**0.5

# initialize the Classifier
logreg = LogisticRegression(max_iter=10000)

# fit the model
logreg.fit(X_train, y_train)    

# make predictions
y_pred = logreg.predict(X_test)

# test the model
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
print(mcc(confusion_matrix(y_test, y_pred)))

# show the classification report
print(classification_report(y_test, y_pred))

In [ ]:
# review_topics = pd.read_csv('../scripts/review_1819_topics_full.csv')
review_topics = pd.read_csv('../scripts/review_1819_topics_train500000_annotfull.csv')

In [ ]:
X = review_topics.filter(like='topic', axis=1)
X.head()

In [ ]:
y = review_topics['useful'].apply(lambda x: 1 if x > 0 else 0)

# split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RSEED, stratify=y)

def mcc(cm):
    tn, fp = cm[0]
    fn, tp = cm[1]
    return (tp*tn-fp*fn) / ((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))**0.5

# initialize the Classifier
logreg = LogisticRegression(max_iter=10000)

# fit the model
logreg.fit(X_train, y_train)    

# make predictions
y_pred = logreg.predict(X_test)

# test the model
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='g')
print(mcc(confusion_matrix(y_test, y_pred)))

# show the classification report
print(classification_report(y_test, y_pred))